In [4]:


# Directory containing the files
dir_path = '229r/all1'

# Directory containing the files
dat_dir_path = 'data/229r/all1'


In [5]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
file_name = """229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt"""

def plot_graph(file_name="""229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt""", title='Average Fitness over Time', width = 800,    height = 600, log_y=True):

    # Read data from file into a list of strings
    with open(file_name, 'r') as f:
        lines = f.readlines()

    # Parse the lines to create a list of dictionaries, where each dictionary contains values for each column
    data = []
    for line in lines:
        if line.startswith('UD:'):
            values = line.strip().split()
            # print(values)

            data.append({
                'UD': int(values[1]),
                'Gen': float(values[3]),
                'Fit': float(values[5]),
                'Orgs': int(values[7])
            })

    # Create dataframe from list of dictionaries
    df = pd.DataFrame(data)
        

    # Define x and y data
    x_data = df.iloc[:, 0] # First column as x-axis
    y_data = df[["Fit"]]
    # y_data = df.iloc[:, 1:] # Every other column as y-axis

    # Create plotly trace for each column
    data = []
    for column in y_data.columns:
        trace = go.Scatter(x=x_data, y=y_data[column], mode='lines', name=column)
        data.append(trace)

    # Define plot layout
    layout = go.Layout(title=title, xaxis=dict(title='# of updates'), yaxis=dict(title='average fitness'), margin=dict(t=40, b=20, l=20, r=20), autosize=False, width=width, height=height, yaxis_type='log' if log_y else 'linear')

    # Create plotly figure
    fig = go.Figure(data=data, layout=layout)

    # Display plot
    file_parts = file_name.split('/')
    fig.show()
    pio.write_image(fig, file_parts[0]+'/plots/'+file_parts[1].split('.')[0]+'.png', width=width, height=height, scale=2)


In [6]:
import os
import pandas as pd
import re
# List to store the file names and last lines
file_data = []

# Regular expression pattern
pattern = r"(\d+)_mxc_([\w-]+)_(\[.*\])_date_(\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_xydict_values\((\[.*\])\)\.txt"
# Create an empty DataFrame with the required columns
column_names = ['index', 'max_count', 'values', 'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit', 'Orgs']
df = pd.DataFrame(columns=column_names)

# Create an empty list to store the data
data_list = []

# Loop through all the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file and read the last line
        with open(os.path.join(dir_path, filename), 'r') as f:
            # Search for pattern in the log_file_name
            match = re.search(pattern, filename)
            if not match:
                raise Exception("No match found for pattern in file name: {}".format(filename))
            index = int(match.group(1))
            max_count = match.group(2)
            if max_count == '-1':
                max_count = False
            elif max_count == '0':
                max_count = True
            else:
                continue
            # max_count = match.group(2) == 'True'
            values = tuple(int(i) for i in match.group(3).strip('[]').split(', '))
            experiment_start_time_string = match.group(4) + "_" + match.group(5)
            xy_values = tuple(int(i) for i in match.group(6).strip('[]').split(', '))
            
            lines = f.readlines()
            last_line = lines[-1].strip()
            
            with open(os.path.join(dat_dir_path, filename), 'r') as fdat: 
                lines_dat = fdat.readlines()
                last_line_dat = lines_dat[-1].strip()
                numbers = last_line_dat.split()

                dictionary = {
                    "Update": int(numbers[0]),
                    "Not": int(numbers[1]),
                    "Nand": int(numbers[2]),
                    "And": int(numbers[3]),
                    "OrNot": int(numbers[4]),
                    "Or": int(numbers[5]),
                    "AndNot": int(numbers[6]),
                    "Nor": int(numbers[7]),
                    "Xor": int(numbers[8])
                }
                EQU_dict = {
                    "Equals": int(numbers[9])}

            
            if last_line.startswith('UD:'):
                last_line_string = last_line.strip().split()

                data = {
                'max_count': max_count,
                'values': values,
                'last_line_string': last_line_string,
                'experiment_start_time_string': experiment_start_time_string,
                'xy_values': xy_values,
                'Last_UD': int(last_line_string[1]),
                'Gen': float(last_line_string[3]),
                'Fit': float(last_line_string[5]),
                'Orgs': int(last_line_string[7]),
                'index': index,
                'filename': filename,
                }
                data.update(dictionary)
                EQU_dict.update(data)

                # Append the data to the list
                data_list.append(EQU_dict)
            else: 
                raise Exception("Last line does not start with UD: {}".format(filename))

# Create the DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the dataframe
df


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,0,False,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 9906.699, Fit:, 1.068264e+9...",04-17_16-23-01,"(120, 120)",10000,9906.699,1.068264e+90,11072,...,"14_mxc_-1_[1, 3, 2, 4, 5]_date_04-17_16-23-01_...",10000,2681,2130,546,4806,16,5,0,0
1,3659,True,"(2, 1, 3, 4, 5)","[UD:, 7500, Gen:, 2688.768, Fit:, 2.278786e+21...",04-18_14-08-24,"(120, 120)",7500,2688.768,2.278786e+216,14243,...,"15_mxc_0_[2, 1, 3, 4, 5]_date_04-18_14-08-24_x...",7500,12625,8530,7777,9272,10479,10959,10183,2208
2,0,False,"(2, 1, 3, 4, 5)","[UD:, 10000, Gen:, 3455.037, Fit:, 2.225637e+2...",04-20_17-28-43,"(120, 120)",10000,3455.037,2.225637e+251,13895,...,"19_mxc_-1_[2, 1, 3, 4, 5]_date_04-20_17-28-43_...",10000,11965,10328,96,1916,18,8750,27,0
3,0,True,"(1, 4, 2, 3, 5)","[UD:, 10000, Gen:, 2139.232, Fit:, 6436.750, O...",04-19_01-01-47,"(120, 120)",10000,2139.232,6.436750e+03,14381,...,"16_mxc_0_[1, 4, 2, 3, 5]_date_04-19_01-01-47_x...",10000,13292,13071,11925,12860,12380,4427,8338,0
4,0,True,"(2, 4, 1, 3, 5)","[UD:, 10000, Gen:, 1971.971, Fit:, 8111.602, O...",04-20_20-27-19,"(120, 120)",10000,1971.971,8.111602e+03,14372,...,"19_mxc_0_[2, 4, 1, 3, 5]_date_04-20_20-27-19_x...",10000,13089,12615,13266,12265,32,157,11552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,3512,True,"(2, 3, 1, 4, 5)","[UD:, 10000, Gen:, 1133.529, Fit:, 4.690366e+1...",04-21_11-55-14,"(120, 120)",10000,1133.529,4.690366e+19,14373,...,"20_mxc_0_[2, 3, 1, 4, 5]_date_04-21_11-55-14_x...",10000,11353,12723,4559,12986,4171,209,4151,0
574,0,False,"(1, 4, 2, 3, 5)","[UD:, 7500, Gen:, 2848.906, Fit:, 2.602341e+26...",04-19_01-21-12,"(120, 120)",7500,2848.906,2.602341e+268,13621,...,"16_mxc_-1_[1, 4, 2, 3, 5]_date_04-19_01-21-12_...",7500,149,954,0,11538,10,27,0,0
575,0,True,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2267.469, Fit:, 4658.941, O...",04-22_01-04-11,"(120, 120)",10000,2267.469,4.658941e+03,14386,...,"21_mxc_0_[1, 2, 3, 4, 1]_date_04-22_01-04-11_x...",10000,13584,13493,145,13380,13002,13042,13036,0
576,0,False,"(1, 2, 4, 3, 5)","[UD:, 9400, Gen:, 3559.430, Fit:, 5.733812e+25...",04-18_10-25-05,"(120, 120)",9400,3559.430,5.733812e+254,14199,...,"15_mxc_-1_[1, 2, 4, 3, 5]_date_04-18_10-25-05_...",9400,399,11795,151,174,7093,11153,0,3


In [7]:
import pandas as pd
import pickle

# Pickle the dataframe to a file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'wb') as f:
    pickle.dump(df, f)


In [8]:
import pandas as pd
import pickle

# Load the pickled dataframe from the file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'rb') as f:
    df = pickle.load(f)

# Print the dataframe
display(df)


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,0,False,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 9906.699, Fit:, 1.068264e+9...",04-17_16-23-01,"(120, 120)",10000,9906.699,1.068264e+90,11072,...,"14_mxc_-1_[1, 3, 2, 4, 5]_date_04-17_16-23-01_...",10000,2681,2130,546,4806,16,5,0,0
1,3659,True,"(2, 1, 3, 4, 5)","[UD:, 7500, Gen:, 2688.768, Fit:, 2.278786e+21...",04-18_14-08-24,"(120, 120)",7500,2688.768,2.278786e+216,14243,...,"15_mxc_0_[2, 1, 3, 4, 5]_date_04-18_14-08-24_x...",7500,12625,8530,7777,9272,10479,10959,10183,2208
2,0,False,"(2, 1, 3, 4, 5)","[UD:, 10000, Gen:, 3455.037, Fit:, 2.225637e+2...",04-20_17-28-43,"(120, 120)",10000,3455.037,2.225637e+251,13895,...,"19_mxc_-1_[2, 1, 3, 4, 5]_date_04-20_17-28-43_...",10000,11965,10328,96,1916,18,8750,27,0
3,0,True,"(1, 4, 2, 3, 5)","[UD:, 10000, Gen:, 2139.232, Fit:, 6436.750, O...",04-19_01-01-47,"(120, 120)",10000,2139.232,6.436750e+03,14381,...,"16_mxc_0_[1, 4, 2, 3, 5]_date_04-19_01-01-47_x...",10000,13292,13071,11925,12860,12380,4427,8338,0
4,0,True,"(2, 4, 1, 3, 5)","[UD:, 10000, Gen:, 1971.971, Fit:, 8111.602, O...",04-20_20-27-19,"(120, 120)",10000,1971.971,8.111602e+03,14372,...,"19_mxc_0_[2, 4, 1, 3, 5]_date_04-20_20-27-19_x...",10000,13089,12615,13266,12265,32,157,11552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,3512,True,"(2, 3, 1, 4, 5)","[UD:, 10000, Gen:, 1133.529, Fit:, 4.690366e+1...",04-21_11-55-14,"(120, 120)",10000,1133.529,4.690366e+19,14373,...,"20_mxc_0_[2, 3, 1, 4, 5]_date_04-21_11-55-14_x...",10000,11353,12723,4559,12986,4171,209,4151,0
574,0,False,"(1, 4, 2, 3, 5)","[UD:, 7500, Gen:, 2848.906, Fit:, 2.602341e+26...",04-19_01-21-12,"(120, 120)",7500,2848.906,2.602341e+268,13621,...,"16_mxc_-1_[1, 4, 2, 3, 5]_date_04-19_01-21-12_...",7500,149,954,0,11538,10,27,0,0
575,0,True,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2267.469, Fit:, 4658.941, O...",04-22_01-04-11,"(120, 120)",10000,2267.469,4.658941e+03,14386,...,"21_mxc_0_[1, 2, 3, 4, 1]_date_04-22_01-04-11_x...",10000,13584,13493,145,13380,13002,13042,13036,0
576,0,False,"(1, 2, 4, 3, 5)","[UD:, 9400, Gen:, 3559.430, Fit:, 5.733812e+25...",04-18_10-25-05,"(120, 120)",9400,3559.430,5.733812e+254,14199,...,"15_mxc_-1_[1, 2, 4, 3, 5]_date_04-18_10-25-05_...",9400,399,11795,151,174,7093,11153,0,3


In [9]:
df.describe()

,Equals,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
count,578.000000,578.000000,578.000000,5.780000e+02,578.000000,578.000000,578.000000,578.000000,578.000000,578.000000,578.000000,578.000000,578.000000,578.000000,578.000000
mean,1833.005190,6519.397924,1663.193550,1.148863e+298,13009.916955,19.655709,6519.031142,8388.432526,8112.685121,3910.031142,7839.410035,5413.899654,5533.247405,4490.749135,718.816609
std,3573.432349,3754.026710,1281.672885,inf,2342.594908,3.392439,3754.274434,5018.280094,5184.303802,5218.896424,5236.033314,5480.201588,5631.203604,5328.384460,2480.888987
min,0.000000,600.000000,46.682100,2.501915e-01,3393.000000,14.000000,600.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2025.000000,467.470475,3.139133e+04,12761.500000,16.000000,2025.000000,3233.750000,2746.250000,5.000000,1792.000000,10.000000,9.000000,0.000000,0.000000
50%,0.000000,8350.000000,1793.716500,1.718598e+24,14236.000000,20.000000,8350.000000,10840.500000,10133.000000,156.500000,10534.500000,2764.500000,2677.000000,139.000000,0.000000
75%,306.000000,10000.000000,2387.734000,1.847496e+190,14362.000000,22.000000,10000.000000,13094.750000,13076.750000,10598.500000,12676.250000,11615.000000,11830.750000,11066.000000,1.000000
max,13372.000000,10000.000000,9906.699000,5.375361e+300,14392.000000,25.000000,10000.000000,14009.000000,13973.000000,13342.000000,13933.000000,13174.000000,13592.000000,13380.000000,11815.000000


In [10]:
df.columns

Index(['Equals', 'max_count', 'values', 'last_line_string',
       'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit',
       'Orgs', 'index', 'filename', 'Update', 'Not', 'Nand', 'And', 'OrNot',
       'Or', 'AndNot', 'Nor', 'Xor'],
      dtype='object')

In [11]:
# Get all the rows with max_count = False and Last_UD = 10000
filtered_df = df[(df['max_count'] == False) & (df['Last_UD'] == 10000)]

In [12]:
filtered_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,0,False,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 9906.699, Fit:, 1.068264e+9...",04-17_16-23-01,"(120, 120)",10000,9906.6990,1.068264e+90,11072,...,"14_mxc_-1_[1, 3, 2, 4, 5]_date_04-17_16-23-01_...",10000,2681,2130,546,4806,16,5,0,0
2,0,False,"(2, 1, 3, 4, 5)","[UD:, 10000, Gen:, 3455.037, Fit:, 2.225637e+2...",04-20_17-28-43,"(120, 120)",10000,3455.0370,2.225637e+251,13895,...,"19_mxc_-1_[2, 1, 3, 4, 5]_date_04-20_17-28-43_...",10000,11965,10328,96,1916,18,8750,27,0
22,0,False,"(2, 4, 1, 3, 5)","[UD:, 10000, Gen:, 3754.978, Fit:, 1.415883e+1...",04-25_07-02-52,"(120, 120)",10000,3754.9780,1.415883e+136,13733,...,"25_mxc_-1_[2, 4, 1, 3, 5]_date_04-25_07-02-52_...",10000,371,11386,643,11452,8,10,6,0
33,0,False,"(1, 3, 4, 2, 5)","[UD:, 10000, Gen:, 2887.565, Fit:, 2.342660e+9...",04-23_15-16-53,"(120, 120)",10000,2887.5650,2.342660e+95,14093,...,"23_mxc_-1_[1, 3, 4, 2, 5]_date_04-23_15-16-53_...",10000,10775,249,0,1635,10937,75,10,0
34,0,False,"(2, 4, 1, 3, 5)","[UD:, 10000, Gen:, 2936.504, Fit:, 3.623644e+2...",04-23_19-42-21,"(120, 120)",10000,2936.5040,3.623644e+264,13938,...,"23_mxc_-1_[2, 4, 1, 3, 5]_date_04-23_19-42-21_...",10000,6048,3514,5,12998,162,30,0,0
38,0,False,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 279.9017, Fit:, 2.368785e+0...",04-21_22-28-55,"(120, 120)",10000,279.9017,2.368785e+07,7809,...,"20_mxc_-1_[4, 3, 2, 1, 5]_date_04-21_22-28-55_...",10000,3321,170,0,1083,10,0,0,0
53,0,False,"(2, 3, 4, 1, 5)","[UD:, 10000, Gen:, 162.1611, Fit:, 4.321074e+2...",04-19_04-19-42,"(120, 120)",10000,162.1611,4.321074e+27,6594,...,"16_mxc_-1_[2, 3, 4, 1, 5]_date_04-19_04-19-42_...",10000,95,1720,0,1608,0,0,0,0
57,0,False,"(2, 1, 4, 3, 5)","[UD:, 10000, Gen:, 989.3356, Fit:, 5.366316e+2...",04-20_19-00-18,"(120, 120)",10000,989.3356,5.366316e+27,14334,...,"19_mxc_-1_[2, 1, 4, 3, 5]_date_04-20_19-00-18_...",10000,10368,10655,141,7613,9366,10616,10,0
60,0,False,"(2, 4, 3, 1, 5)","[UD:, 10000, Gen:, 191.7533, Fit:, 1.612656e+1...",04-25_07-44-40,"(120, 120)",10000,191.7533,1.612656e+12,6429,...,"25_mxc_-1_[2, 4, 3, 1, 5]_date_04-25_07-44-40_...",10000,1667,33,0,1633,2,0,0,0
62,0,False,"(4, 2, 1, 3, 5)","[UD:, 10000, Gen:, 2491.899, Fit:, 1.226074e+2...",04-20_07-28-30,"(120, 120)",10000,2491.8990,1.226074e+268,14128,...,"18_mxc_-1_[4, 2, 1, 3, 5]_date_04-20_07-28-30_...",10000,12473,12711,13,207,0,0,0,0


In [13]:
sorted_EQU_df = df.sort_values("Equals", ascending=False)

In [14]:
sorted_EQU_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
514,13372,True,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 3333.767, Fit:, 9.435327e+0...",04-17_16-05-58,"(120, 120)",10000,3333.76700,9.435327e+09,14234,...,"14_mxc_0_[1, 3, 2, 4, 5]_date_04-17_16-05-58_x...",10000,14009,13303,12028,13068,12673,13388,13349,16
109,12262,True,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 3144.089, Fit:, 2.497056e+1...",04-23_09-52-11,"(120, 120)",10000,3144.08900,2.497056e+18,14134,...,"22_mxc_0_[4, 3, 2, 1, 5]_date_04-23_09-52-11_x...",10000,13364,13083,12682,13388,9921,12503,850,1
66,11738,True,"(4, 2, 1, 3, 5)","[UD:, 10000, Gen:, 2259.783, Fit:, 42466.73, O...",04-20_07-13-26,"(120, 120)",10000,2259.78300,4.246673e+04,14369,...,"18_mxc_0_[4, 2, 1, 3, 5]_date_04-20_07-13-26_x...",10000,12619,13757,12375,13009,8,487,28,6
36,11712,True,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 3086.236, Fit:, 1.864469e+1...",04-18_10-54-13,"(120, 120)",10000,3086.23600,1.864469e+162,14068,...,"15_mxc_0_[1, 3, 2, 4, 5]_date_04-18_10-54-13_x...",10000,1902,9888,12265,12977,11152,11663,10411,181
495,11671,True,"(4, 2, 1, 3, 5)","[UD:, 5900, Gen:, 3480.698, Fit:, 9.543012e+21...",04-21_20-12-35,"(120, 120)",5900,3480.69800,9.543012e+213,14285,...,"20_mxc_0_[4, 2, 1, 3, 5]_date_04-21_20-12-35_x...",5900,1756,1861,1102,1473,441,286,11142,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,0,True,"(4, 1, 3, 2, 5)","[UD:, 3575, Gen:, 469.0538, Fit:, 750.6426, Or...",04-24_00-49-41,"(120, 120)",3575,469.05380,7.506426e+02,14337,...,"23_mxc_0_[4, 1, 3, 2, 5]_date_04-24_00-49-41_x...",3500,12652,13077,62,317,5707,4928,1,5
220,0,False,"(1, 3, 2, 4, 5)","[UD:, 1500, Gen:, 128.9594, Fit:, 5.720062e+14...",04-22_22-52-33,"(120, 120)",1500,128.95940,5.720062e+141,12252,...,"22_mxc_-1_[1, 3, 2, 4, 5]_date_04-22_22-52-33_...",1500,1549,554,2,1243,97,18,40,0
219,0,False,"(2, 4, 1, 3, 5)","[UD:, 2100, Gen:, 629.4133, Fit:, 4.431596e+14...",04-24_09-22-33,"(120, 120)",2100,629.41330,4.431596e+149,12145,...,"24_mxc_-1_[2, 4, 1, 3, 5]_date_04-24_09-22-33_...",2100,88,703,298,7976,8,3,0,0
217,0,False,"(4, 2, 3, 1, 5)","[UD:, 700, Gen:, 55.34626, Fit:, 0.3000963, Or...",04-21_03-06-12,"(120, 120)",700,55.34626,3.000963e-01,6859,...,"19_mxc_-1_[4, 2, 3, 1, 5]_date_04-21_03-06-12_...",700,93,0,0,0,0,0,0,0


In [15]:
cleaned_df = sorted_EQU_df[sorted_EQU_df['Last_UD'] == 10000]
display(cleaned_df)

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
514,13372,True,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 3333.767, Fit:, 9.435327e+0...",04-17_16-05-58,"(120, 120)",10000,3333.767,9.435327e+09,14234,...,"14_mxc_0_[1, 3, 2, 4, 5]_date_04-17_16-05-58_x...",10000,14009,13303,12028,13068,12673,13388,13349,16
109,12262,True,"(4, 3, 2, 1, 5)","[UD:, 10000, Gen:, 3144.089, Fit:, 2.497056e+1...",04-23_09-52-11,"(120, 120)",10000,3144.089,2.497056e+18,14134,...,"22_mxc_0_[4, 3, 2, 1, 5]_date_04-23_09-52-11_x...",10000,13364,13083,12682,13388,9921,12503,850,1
66,11738,True,"(4, 2, 1, 3, 5)","[UD:, 10000, Gen:, 2259.783, Fit:, 42466.73, O...",04-20_07-13-26,"(120, 120)",10000,2259.783,4.246673e+04,14369,...,"18_mxc_0_[4, 2, 1, 3, 5]_date_04-20_07-13-26_x...",10000,12619,13757,12375,13009,8,487,28,6
36,11712,True,"(1, 3, 2, 4, 5)","[UD:, 10000, Gen:, 3086.236, Fit:, 1.864469e+1...",04-18_10-54-13,"(120, 120)",10000,3086.236,1.864469e+162,14068,...,"15_mxc_0_[1, 3, 2, 4, 5]_date_04-18_10-54-13_x...",10000,1902,9888,12265,12977,11152,11663,10411,181
58,11631,True,"(3, 4, 1, 2, 5)","[UD:, 10000, Gen:, 2287.056, Fit:, 1299638., O...",04-20_05-14-45,"(120, 120)",10000,2287.056,1.299638e+06,14349,...,"18_mxc_0_[3, 4, 1, 2, 5]_date_04-20_05-14-45_x...",10000,13033,13298,844,13653,12166,57,12250,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,True,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2566.778, Fit:, 17122.64, O...",04-21_06-33-46,"(120, 120)",10000,2566.778,1.712264e+04,14381,...,"20_mxc_0_[1, 2, 3, 4, 1]_date_04-21_06-33-46_x...",10000,13468,12707,12835,12922,12445,13468,12205,0
240,0,False,"(1, 2, 3, 4, 2)","[UD:, 10000, Gen:, 4970.538, Fit:, 2.763625e+4...",04-25_00-26-30,"(120, 120)",10000,4970.538,2.763625e+45,14054,...,"25_mxc_-1_[1, 2, 3, 4, 2]_date_04-25_00-26-30_...",10000,10906,251,0,11177,11206,36,8197,0
229,0,False,"(1, 2, 3, 4, 3)","[UD:, 10000, Gen:, 5236.037, Fit:, 1.478528e+2...",04-24_02-24-47,"(120, 120)",10000,5236.037,1.478528e+250,14065,...,"24_mxc_-1_[1, 2, 3, 4, 3]_date_04-24_02-24-47_...",10000,4655,7874,153,10345,6125,6884,3717,0
226,0,True,"(1, 2, 3, 4, 1)","[UD:, 10000, Gen:, 2243.788, Fit:, 15955.55, O...",04-23_12-39-18,"(120, 120)",10000,2243.788,1.595555e+04,14375,...,"23_mxc_0_[1, 2, 3, 4, 1]_date_04-23_12-39-18_x...",10000,13476,13180,12728,13531,11513,12377,11670,0


In [16]:
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[cleaned_df["max_count"]]
d1 = d1[["Equals", "Fit"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &        Equals &            Fit \\
\midrule
mean &   2587.576923 &  2.652688e+264 \\
min  &      0.000000 &   3.307935e+02 \\
25\%  &      0.000000 &   1.369368e+04 \\
50\%  &      0.000000 &   4.006197e+04 \\
75\%  &   5830.500000 &   1.090476e+06 \\
max  &  13372.000000 &  5.517592e+266 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_1044/2916242337.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [17]:
# display(cleaned_df[~cleaned_df["max_count"]].describe())
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[~cleaned_df["max_count"]]
d1 = d1[["Equals", "Fit", "filename"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &       Equals &            Fit \\
\midrule
mean &    54.472727 &  7.225835e+292 \\
min  &     0.000000 &   2.368785e+07 \\
25\%  &     0.000000 &   5.559675e+69 \\
50\%  &     0.000000 &   1.068264e+90 \\
75\%  &     0.000000 &  4.060555e+186 \\
max  &  2799.000000 &  3.974209e+294 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_1044/652201068.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [18]:
cleaned_df = cleaned_df[["values", "max_count", "Equals", "Fit", "index", "filename"]]

In [19]:
diff_vals_df = cleaned_df.sort_values("values")

In [20]:
print(diff_vals_df.to_latex(index=False))

/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_1044/128294827.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(diff_vals_df.to_latex(index=False))


\begin{tabular}{llrrrl}
\toprule
         values &  max\_count &  Equals &           Fit &  index &                                           filename \\
\midrule
(1, 2, 3, 4, 1) &       True &       0 &  1.051585e+04 &     18 & 18\_mxc\_0\_[1, 2, 3, 4, 1]\_date\_04-19\_20-18-53\_xy... \\
(1, 2, 3, 4, 1) &       True &       9 &  2.043470e+05 &     22 & 22\_mxc\_0\_[1, 2, 3, 4, 1]\_date\_04-22\_17-19-10\_xy... \\
(1, 2, 3, 4, 1) &       True &       0 &  1.595555e+04 &     23 & 23\_mxc\_0\_[1, 2, 3, 4, 1]\_date\_04-23\_12-39-18\_xy... \\
(1, 2, 3, 4, 1) &       True &       0 &  2.875752e+03 &     25 & 25\_mxc\_0\_[1, 2, 3, 4, 1]\_date\_04-25\_00-51-17\_xy... \\
(1, 2, 3, 4, 1) &      False &       0 & 8.290687e+108 &     16 & 16\_mxc\_-1\_[1, 2, 3, 4, 1]\_date\_04-18\_21-36-55\_x... \\
(1, 2, 3, 4, 1) &      False &       0 &  5.570365e+47 &     14 & 14\_mxc\_-1\_[1, 2, 3, 4, 1]\_date\_04-17\_14-25-59\_x... \\
(1, 2, 3, 4, 1) &       True &       0 &  3.825357e+03 &     24 & 24\_mxc\_

In [21]:
display(diff_vals_df)

,values,max_count,Equals,Fit,index,filename
556,"(1, 2, 3, 4, 1)",True,0,1.051585e+04,18,"18_mxc_0_[1, 2, 3, 4, 1]_date_04-19_20-18-53_x..."
191,"(1, 2, 3, 4, 1)",True,9,2.043470e+05,22,"22_mxc_0_[1, 2, 3, 4, 1]_date_04-22_17-19-10_x..."
226,"(1, 2, 3, 4, 1)",True,0,1.595555e+04,23,"23_mxc_0_[1, 2, 3, 4, 1]_date_04-23_12-39-18_x..."
116,"(1, 2, 3, 4, 1)",True,0,2.875752e+03,25,"25_mxc_0_[1, 2, 3, 4, 1]_date_04-25_00-51-17_x..."
78,"(1, 2, 3, 4, 1)",False,0,8.290687e+108,16,"16_mxc_-1_[1, 2, 3, 4, 1]_date_04-18_21-36-55_..."
...,...,...,...,...,...,...
215,"(4, 3, 2, 1, 5)",True,0,4.886765e+04,20,"20_mxc_0_[4, 3, 2, 1, 5]_date_04-21_22-10-55_x..."
38,"(4, 3, 2, 1, 5)",False,0,2.368785e+07,20,"20_mxc_-1_[4, 3, 2, 1, 5]_date_04-21_22-28-55_..."
109,"(4, 3, 2, 1, 5)",True,12262,2.497056e+18,22,"22_mxc_0_[4, 3, 2, 1, 5]_date_04-23_09-52-11_x..."
212,"(4, 3, 2, 1, 5)",True,0,7.765631e+04,18,"18_mxc_0_[4, 3, 2, 1, 5]_date_04-20_08-59-32_x..."


In [22]:
fdf = diff_vals_df 

In [23]:
fdf

,values,max_count,Equals,Fit,index,filename
556,"(1, 2, 3, 4, 1)",True,0,1.051585e+04,18,"18_mxc_0_[1, 2, 3, 4, 1]_date_04-19_20-18-53_x..."
191,"(1, 2, 3, 4, 1)",True,9,2.043470e+05,22,"22_mxc_0_[1, 2, 3, 4, 1]_date_04-22_17-19-10_x..."
226,"(1, 2, 3, 4, 1)",True,0,1.595555e+04,23,"23_mxc_0_[1, 2, 3, 4, 1]_date_04-23_12-39-18_x..."
116,"(1, 2, 3, 4, 1)",True,0,2.875752e+03,25,"25_mxc_0_[1, 2, 3, 4, 1]_date_04-25_00-51-17_x..."
78,"(1, 2, 3, 4, 1)",False,0,8.290687e+108,16,"16_mxc_-1_[1, 2, 3, 4, 1]_date_04-18_21-36-55_..."
...,...,...,...,...,...,...
215,"(4, 3, 2, 1, 5)",True,0,4.886765e+04,20,"20_mxc_0_[4, 3, 2, 1, 5]_date_04-21_22-10-55_x..."
38,"(4, 3, 2, 1, 5)",False,0,2.368785e+07,20,"20_mxc_-1_[4, 3, 2, 1, 5]_date_04-21_22-28-55_..."
109,"(4, 3, 2, 1, 5)",True,12262,2.497056e+18,22,"22_mxc_0_[4, 3, 2, 1, 5]_date_04-23_09-52-11_x..."
212,"(4, 3, 2, 1, 5)",True,0,7.765631e+04,18,"18_mxc_0_[4, 3, 2, 1, 5]_date_04-20_08-59-32_x..."


In [24]:
gdf = diff_vals_df.groupby(["values", "max_count"]).mean()
gdf

Equals            Fit      index
values          max_count                                       
(1, 2, 3, 4, 1) False         0.250000  2.073168e+108  18.000000
                True          2.090909   4.265361e+04  19.727273
(1, 2, 3, 4, 2) False         0.000000  2.580365e+144  19.500000
                True        122.200000   3.499869e+04  19.300000
(1, 2, 3, 4, 3) False         0.000000  7.392640e+249  20.000000
                True       3584.625000   1.585436e+06  20.750000
(1, 2, 3, 4, 4) False        97.000000  2.742304e+119  16.500000
                True        958.500000   1.873431e+37  19.300000
(1, 2, 3, 4, 5) False         0.250000  3.639272e+203  20.000000
                True       1820.333333   3.466783e+20  21.500000
(1, 2, 3, 4, 6) False         0.000000   2.248926e+14  16.000000
                True       2550.333333  2.488017e+122  20.166667
(1, 2, 4, 3, 5) False         0.000000  7.504320e+191  19.750000
                True       3804.500000  6.770731e+182  20.875000
(1, 3, 2, 4, 5) False         0.000000  5.420650e+274  21.000000
                True       3629.800000  1.864469e+161  19.400000
(1, 3, 4, 2, 5) False         0.000000  8.357887e+172  21.000000
                True       1827.500000   7.501531e+04  19.500000
(1, 4, 2, 3, 5) False      1400.000000  1.886180e+251  23.000000
                True          2.000000   1.384294e+04  18.666667
(1, 4, 3, 2, 5) True       1523.375000  1.993654e+125  19.375000
(2, 1, 3, 4, 5) False         0.000000  8.635653e+264  18.000000
                True       2097.800000  3.681372e+111  22.400000
(2, 1, 4, 3, 5) False         0.000000   1.164531e+40  16.000000
                True       5123.750000  8.951434e+227  19.750000
(2, 3, 1, 4, 5) False         0.000000  4.060554e+186  21.500000
                True       1943.222222   5.211554e+18  19.888889
(2, 3, 4, 1, 5) False         0.000000   4.321074e+27  16.000000
                True       3064.000000  1.003080e+116  20.000000
(2, 4, 1, 3, 5) False         0.000000  1.324736e+294  22.333333
                True       4917.375000  2.710946e+241  19.000000
(2, 4, 3, 1, 5) False         0.000000   2.110377e+86  24.000000
                True       1222.571429   4.279266e+05  19.285714
(3, 1, 2, 4, 5) False         0.000000  5.319570e+266  21.000000
                True        894.000000   1.163402e+59  20.500000
(3, 1, 4, 2, 5) False         0.000000   9.666460e+69  23.000000
                True       5724.333333  9.765522e+157  20.500000
(3, 2, 1, 4, 5) True       2454.333333   3.042773e+77  20.666667
(3, 2, 4, 1, 5) True       1089.166667   9.720889e+04  19.000000
(3, 4, 1, 2, 5) False         0.000000   4.437111e+19  20.000000
                True       4695.285714   2.527464e+99  20.285714
(3, 4, 2, 1, 5) False         0.000000   2.518244e+21  25.000000
                True       3385.666667   3.187479e+35  19.777778
(4, 1, 2, 3, 5) False         0.000000  1.331295e+102  16.000000
                True       2807.500000   8.599064e+04  18.750000
(4, 1, 3, 2, 5) True       2962.428571  9.671359e+131  18.571429
(4, 2, 1, 3, 5) False         0.000000  6.130370e+267  16.000000
                True       5061.833333  2.046535e+214  18.333333
(4, 2, 3, 1, 5) True       3584.800000  1.103518e+266  19.000000
(4, 3, 1, 2, 5) False         0.000000  1.585746e+223  18.000000
                True       1687.500000   1.277967e+09  18.666667
(4, 3, 2, 1, 5) False         0.000000   2.368785e+07  20.000000
                True       4690.000000  3.599086e+186  19.000000

In [25]:
gdf = gdf.reset_index().sort_values("Equals")

In [26]:
gdf[gdf["max_count"] == False]

,values,max_count,Equals,Fit,index
12,"(1, 2, 4, 3, 5)",False,0.00,7.504320e+191,19.750000
23,"(2, 1, 4, 3, 5)",False,0.00,1.164531e+40,16.000000
41,"(3, 4, 2, 1, 5)",False,0.00,2.518244e+21,25.000000
21,"(2, 1, 3, 4, 5)",False,0.00,8.635653e+264,18.000000
51,"(4, 3, 2, 1, 5)",False,0.00,2.368785e+07,20.000000
27,"(2, 3, 4, 1, 5)",False,0.00,4.321074e+27,16.000000
33,"(3, 1, 2, 4, 5)",False,0.00,5.319570e+266,21.000000
16,"(1, 3, 4, 2, 5)",False,0.00,8.357887e+172,21.000000
43,"(4, 1, 2, 3, 5)",False,0.00,1.331295e+102,16.000000
14,"(1, 3, 2, 4, 5)",False,0.00,5.420650e+274,21.000000


In [27]:
gdf[gdf["max_count"] == True]

,values,max_count,Equals,Fit,index
19,"(1, 4, 2, 3, 5)",True,2.000000,1.384294e+04,18.666667
1,"(1, 2, 3, 4, 1)",True,2.090909,4.265361e+04,19.727273
3,"(1, 2, 3, 4, 2)",True,122.200000,3.499869e+04,19.300000
34,"(3, 1, 2, 4, 5)",True,894.000000,1.163402e+59,20.500000
7,"(1, 2, 3, 4, 4)",True,958.500000,1.873431e+37,19.300000
38,"(3, 2, 4, 1, 5)",True,1089.166667,9.720889e+04,19.000000
32,"(2, 4, 3, 1, 5)",True,1222.571429,4.279266e+05,19.285714
20,"(1, 4, 3, 2, 5)",True,1523.375000,1.993654e+125,19.375000
50,"(4, 3, 1, 2, 5)",True,1687.500000,1.277967e+09,18.666667
9,"(1, 2, 3, 4, 5)",True,1820.333333,3.466783e+20,21.500000


In [28]:
gdf[(gdf["max_count"] == True) & (gdf['values'].apply(lambda tup: tup[0] == 1))]

,values,max_count,Equals,Fit,index
19,"(1, 4, 2, 3, 5)",True,2.000000,1.384294e+04,18.666667
1,"(1, 2, 3, 4, 1)",True,2.090909,4.265361e+04,19.727273
3,"(1, 2, 3, 4, 2)",True,122.200000,3.499869e+04,19.300000
7,"(1, 2, 3, 4, 4)",True,958.500000,1.873431e+37,19.300000
20,"(1, 4, 3, 2, 5)",True,1523.375000,1.993654e+125,19.375000
9,"(1, 2, 3, 4, 5)",True,1820.333333,3.466783e+20,21.500000
17,"(1, 3, 4, 2, 5)",True,1827.500000,7.501531e+04,19.500000
11,"(1, 2, 3, 4, 6)",True,2550.333333,2.488017e+122,20.166667
5,"(1, 2, 3, 4, 3)",True,3584.625000,1.585436e+06,20.750000
15,"(1, 3, 2, 4, 5)",True,3629.800000,1.864469e+161,19.400000


In [29]:
gdf[(gdf["max_count"] == True) & (gdf['values'].apply(lambda tup: tup[0] == 4))]

,values,max_count,Equals,Fit,index
50,"(4, 3, 1, 2, 5)",True,1687.500000,1.277967e+09,18.666667
44,"(4, 1, 2, 3, 5)",True,2807.500000,8.599064e+04,18.750000
45,"(4, 1, 3, 2, 5)",True,2962.428571,9.671359e+131,18.571429
48,"(4, 2, 3, 1, 5)",True,3584.800000,1.103518e+266,19.000000
52,"(4, 3, 2, 1, 5)",True,4690.000000,3.599086e+186,19.000000
47,"(4, 2, 1, 3, 5)",True,5061.833333,2.046535e+214,18.333333


In [30]:
fdf[fdf["values"] == (4, 1, 2, 3, 5)]

,values,max_count,Equals,Fit,index,filename
577,"(4, 1, 2, 3, 5)",True,0,2.624929e+04,14,"14_mxc_0_[4, 1, 2, 3, 5]_date_04-18_03-42-57_x..."
134,"(4, 1, 2, 3, 5)",True,0,8.989186e+03,20,"20_mxc_0_[4, 1, 2, 3, 5]_date_04-21_19-08-19_x..."
353,"(4, 1, 2, 3, 5)",True,11230,1.299495e+05,19,"19_mxc_0_[4, 1, 2, 3, 5]_date_04-21_01-28-48_x..."
145,"(4, 1, 2, 3, 5)",True,0,1.787746e+05,22,"22_mxc_0_[4, 1, 2, 3, 5]_date_04-23_07-12-08_x..."
337,"(4, 1, 2, 3, 5)",False,0,1.331295e+102,16,"16_mxc_-1_[4, 1, 2, 3, 5]_date_04-19_08-42-57_..."


In [31]:
fname = fdf[fdf["values"] == (4, 1, 2, 3, 5)]["filename"].iloc[0]

In [32]:
plot_graph(file_name=dir_path + "/" + fname)

In [33]:
def plot_vals(vals, max_count, log_y = True, i = 0):
    fname = fdf[(fdf["values"] == vals) & (fdf["max_count"] == max_count)]["filename"].iloc[i]
    plot_graph(file_name=dir_path + "/" + fname, title=f"""Average Fitness over Time for Reward Configuration {vals} with Max-Count {'On' if max_count else 'Off'}""", log_y=log_y)

In [34]:
plot_vals((1, 2, 3, 4, 5), max_count=True, i=2)

In [35]:
plot_vals((1, 2, 3, 4, 5), max_count=False, i=2)

In [36]:
plot_vals((4, 2, 3, 1, 5), max_count=True, log_y=True, i=0)

In [37]:
plot_vals((4, 2, 3, 1, 5), max_count=False, log_y=True, i=1)

IndexError: single positional indexer is out-of-bounds